# AST 245 Computational Astrophysics N-Body Project

### Libraries

```numba```: optimized running with decorater ```@jit(nopython=True)```, can also easily be parallelized with ```@jit(nopython=True, parallel=True)``` and using ```prange``` instead of ```range``` in the forloop -> careful: race condtion

```numpy```: allows vectorized calculations, improves performance as well

```matplotlib```: for plotting

```scipy.optimize```: from this library use the function ```curve_fit``` to fit a curve

### Data extraction

Kinda self explanatory

### Task 1: Step 1

Calculate a density profile and compare it with the density given from the Hernquist profile

To calculate the expected density for our data for a certain radius, use the calculation from Hernquist (1990):
$\rho(r) = \frac{M}{2 \pi} \frac{a}{r} \frac{1}{(r+a)^3}$, where *M* is the total mass and *a* is the scale length. We are given *M* and *r*, but *a* has to be fitted on the system with the data given and ```curve_fit```.

First, get the desired bins, the absolute radii of the particles, and the widths of the bins. As the data is distributed logarithmically, the bins also have to be created in logspace. This can be done with ```np.logspace```. This takes as start and stop value the exponent that is then applied to the given base, therefore, to get the appropriate value to insert, calculate the exponent of the minimum and tha maximum of the absolute radii of the particles. With that, the bins (which will then be the bin edges) can be calculated. The calculated widths is for plotting the histogram in the end.

Then, calculate the number of particles per bin and the bin edges (these should be the same as the bins). With the prepared bins (= bin edges and the absolute radii of the particles) the particles can now be split across the bins with ```np.histogram```, which returns the particles per bin and the bin edges again (as a sequence was given).

Then calculate the masses enclosed in each shell. As all particles have the same mass, the particles per bin calculated before can be multiplied by the mass of the particle.

Then calculate the volume of each shell / bin. The formula for the shell volume is $V = \frac{4}{3} * \pi * (r_{outer}^3 - r_{inner}^3)$. This can be calculated by the pairwise subtraction of the bin edges.

Then calculate the density in each shell with the formula $\rho = \frac{mass_{shell}}{Volume_{shell}$.

Calculate the absolute and relative error of the distribution of the density in the bins. This is questionable and I'm not sure if this is how it should be, but it works, even with including the absolute in the fitting of the Hernquist profile.

Use this to then fit the Hernquist profile to the data to calculate the expected density per bin (get also the average radius for each bin for plotting). As some radius is required as input, the average radius of each bin is taken, which is then associated with the expected density in said bin. The total mass is calculated and made into an array of the same length as ```bin_radii```, so that ```numpy``` can work with it. An inital guess is made, as otherwise the guess for the fitted parameter *a* will be way off and not achieved in the given circulations. The parameter *a* and the covariance are then estimated with the ```scipy.curve_fit``` function. The fitted *a* is then used in the ```hernquist_rho``` function to calculate the expected number density.

Plot everything.

The received plot is scaled with ```log```. Our distribution fits rather nicely with the Hernquist profile. The explanation for the lack of error bars in the middle comes from the number of particles in the different areas in the distribution. At the innermost volumes and the outermost volumes, we only have few particles, for the innermost still a high density, because the bins are logarithmic, therefore there is a small volume for these particles, while towards middle, more particles occupy a bigger volume. The outermost volumes contain again only a few particles, but this time across a bigger space, therefore there is a low density.

##### Units

In N-body problems, typical units of length is the **parsec** and for mass the **solar mass**, therefore these were chosen for the system. To then calculate the unit for the time, the following equation can be used: $T_0 = \sqrt{\frac{R_0^3}{G * M_O}}$, where all the units are inserted: $3.0857 * 10^{16} m$ for pc, $1.98847 * 10^{30} kg$ for the solar mass, $6.6743 * 10^-11 \frac{m^3}{kg * s^2}$ for G. When this is solved, it gives $4.7 * 10^{14} s$, which is $14.91 Myr$, giving us the unit for time as **Myr**.

### Task 1: Step 2

- Compute directly the forces for all the particles
- Start with assuming a softening of the order of the mean interparticle distance
- Repeat calculation with different orders of magnitude for the softening

The formula for the direct calculation of the force is $F(r) = a * m$, where $a = -G \sum_{j=1}^{N} \frac{m_j}{[(r_i - r_j)^2 + \epsilon^2]^{3/2}} (r_i - r_j)$. To get the total force in the end, calculate this as $F_{abs} = \sqrt{F_x^2 + F_y^2 + F_z^2}$. For the calculation, $G=1$. To improve performance, the outer loop of the calculation is parallelized, but the inner is not to avoid race conditions.

The softening parameter $\epsilon$ is added to the brute force calculation. To calculate the epsilon, that is used for a specific iteration of the direct force calculation:
- first the half mass radius is calculated. This is the radius, that contains half the mass of the system. For this, first the half mass is calculated as $\frac{\text{total mass of the system}}{2}$. Then, from the sorted absolute radii of the particles, the index of the radius, that contains half the mass of the system is determined, which is then the half-mass radius.
- based on this radius, it is determined, which particles are contained within this radius. For this, a boolean mask is generated, that is a array of the same length as the (unsorted) absolute radii, that contains ```True``` if the particle's radius is within the half-mass radius, and ```False``` otherwise. This is then applied on the positions (x, y, z) of the particles.
- Based on this, the mean interparticle distance of the particles contained in the half-mass radius can be calculated, which is the sum of the distances for each particle to the other particles, divided by the number of interactions (which is the number of particles times the number of particles that influence each particle).
- As we should repeat the direct force calculation for several softening parameters, this mean interparticle distance is then used in a function, that also accepts an exponent for 10 and a step size. From this, a list of softening parameters is generated, with the lowest value $softening_{parameter} * (10^{-exp})$ and the highest value $softening_{parameter} * (10^{exp})$, with $stepsize$ steps.

This is then used in the function to generate a plot, for which the direct force calculated as often as there are values in the list of *softening parameters*. The absolute forces and the absolute radii are then sorted together with ```np.rec.fromarrays```.

**Interpretation of the plot**
Gravitational softening is introduced into the model to avoid that the system becomes not collisionless. As the softening is added to the radius, it gives the calculation a minimal radius that it has to work with, the particles are restricted in their movement. Without gravitational softening, particles that get very close to each other, will scatter widely. When the distance between two particles is much bigger than $\epsilon$, the calculation for the force still approaches the regular force, if the distance is much smaller, the force is a constant.
Small softenings are more accurate for the force.
Large softenings lead to a smoothenings at the beginning, though when they become very large, the forces do some fun stuff.

##### Shell Theorem

To check the difference the softening makes, the forces calculated directly with different softenings are compared with the analytical solution calculated on the base of Newton's second shell theorem.

Newton's First Shell Theorem: *A body that is inside a spherical shell of matter experiences no net gravitational force from that shell.*

Newton's Second Shell Theorem: *The gravitational force on a body that lies outside a spherical shell of matter is the same as it would be if all the shell's matter were concentrated into a point at its centre.*

 According to Galactic Dynamics (2008) p. 62: *"From Newton's first and second theorems, if follows that the gravitational attraction of a spherical density distribution $\rho(r')$ on a unit mass at radius r is entirely determined by the mas interior to r"*: $F(r) = - \frac{GM(r)}{r^2} * ê_r$ where $M(r) = 4 \pi \int_0^r dr' r'^2 \rho(r')$
 
To solve this, I first split the volume of the particle into shells. For this I calculate the absolute radii of each particle and the maximum thereof. This maximal radius (which also determines the maximal sphere) is then used in ```np.logspace``` to divide it all in even spheres and also get the shell_thickness.

With ```in_shell = np.where((radii >= lower_bound) & (radii <= upper_bound))``` an array is created that contains an array for each shell, and in this shell are the indices of the particles in said shell. To return this as one array for other functions, the arrays are made the same length by padding with -1 as 0 would be an index and NAN makes a ton of problems. The arrays are then filled with the indices of the particle that are in that shell.

Then the density $\rho$ for each shell is calculated. First, the mass of each shell is determined by summing the masses of all the particles in a shell. Then the volume is calculated by subtracting the volume of the inner sphere (determined by the inner shell radius) from the volume of the outer sphere (determined by the outer shell). The density of each shell is then calculated by dividing the mass of each shell by its volume.

Then the masses for the shells should be calculated, according to this upper integral. First, the density for each shell is calculated. Then the integral is calculated as the sum of these density masses up to, but not including the radius of the shell the particle is in. This gives an approximation, it is not exact.

Finally, the forces are calculated and the absolute forces and the absolute radii are returned and can be added to the plot, after they have been sorted accordingly.

As it is an approximation, for the forces spaced closely together at the innermost radii, the calculation is not exact, but the approximation assimilates to the direct summation.

##### Computation of Relaxation Time

First, the typical particle velocity has to be calculated with the formula $v = \sqrt{\frac{G * M(R_{hm})}{R_{hm}}$, where $M(R_{hm})$ is the mass enclosed by the half mass radius.

Then, the crossing time is calculated with the formula $t_{cross} = \frac{R_{hm}}{v}$.

From this, the relaxation time is then calculated according to the formula $t_{relax} = \frac{N}{8 * \log{N}} * t_{cross}$.

The relaxation time is the time, when the system is not collisionless anymore, where particles interact crazily with each other. They get close to one another and deflect widely due to the strong acceleration / force.
With a large softening, this kind of behaviour is avoided, as less interactions are permitted.
Therefore, an increased softening leads to a longer relaxation time.
In the derivation for the relaxation time, the softening takes influence at the $b_{min}$ of the Coulomb Integral. It is usually chosen where very strong deflections happen, but with the softening, this is restricted to the softening, it limits these strong interactions.

### Task 2: Tree Code

A class point is constructed for the particles. With this, each particle has an x-coordinate, y-coordinate, z-coordinate, a mass, an identification, and an acceleration in x, y, and z direction.

The class ```OctTreeNode``` builds the tree and also creates the nodes, that contain the particles. In the constructer, the coordinates of the bounding box are stored as min and max coordinates for each axis. Each node also contains information about its midpoint for each axis. Further, each node has a total mass and the coordinates of the center of mass associated with it. Two lists are initiated, one to store the particles in the node, the other to store the children of the node. The node has also a limit for the number of particles that can be stored in it before the node is split into children. The value theta will be needed as the threshold. s is the side length of the node.

```add_point``` checks if the current node is already full. If it is already full, the node is split. It then checks if it is an external node (```self.points is not None```, ```None``` marks and internal node). If it is external, the point is added to the node and the center of mass of that node due to that point is calculated. If it is an internal node, the point is added to one of its children.

```split_node``` splits the current node into 8 children according to the min, mid, and max values of the current node. The current points that were already in the node, get assigned a new node (child node). Then, ```self.points``` gets set to ```None``` to mark it as an internal node.

```add_point_to_child``` loops over the children in the node and checks where the node fits and adds the point in this child node. As soon as a fitting node is found, the loop breaks, as a particle can not be in two nodes. When the point has been added to a child, the center of mass is updated.

```calculate_center_of_mass``` calculates the center of mass of an external node with the mass and the coordinates of the point.

```update_center_of_mass``` updates the center of mass from the child node.

```calculate_acceleration``` takes a point and walks the tree to check how the particle has to be treated to calculate the acceleration. Check if there is a point in ```self.points``` (means it's an external node). If that is the case, classically calculate the acceleration the particle experiences from this point. 
If it is not an external node, calculate the distance from the nodes center of mass. Check if the threshold holds and if it holds, treat the node as one thing, and use the total mass and the center of mass for the acceleration on the current particle.
If the threshold does not hold, go recursively through the children and calculate their acceleration.
The acceleration is then assigned to each particle (instance of class)

To check out different angles, the function ```opening_angles``` takes a start and stop value and a number of angles to get an array of different values for the angle.

Then the forces are calculated. For the bounding box, the min and max values of the absolute radii are taken, so that the box has a quadratic shape. The octtree is initiated, then the points are prepared, so they have the correct format. The there is a loop over each point to add it to the tree. In a second loop, the points get looped over again, and for each the acceleration is calculated.

To get the absolute forces, each vector component is multiplied by the mass, and then the absolute force is calculated.

With ```repetitive_tree_code``` is used to run the tree code several times with different values for *theta" and then plot it.


##### Computational cost

```time.perf_counter``` is used to stop the time needed for the force calculations. For the direct force calculation, the time needed varies between 60 seconds and ~30 seconds, depending on if it is a newly started session or not. That it is so fast is due to the acceleration with ```jit```. The time elapsed does not change with the value of the softening

For the tree code, the time needed varies with the change in the angle (rather long for small ones), as with a large opening angle, more nodes are treated as one thing and not the children or particles are looked at. If the angle goes towards 0, more particles have to be looked at individually, going to a direct force calculation for theta = 0.

That the tree code does not perform much better than the direct summation is due to the acceleration of the direct summation with ```jit```, which was not possible for the tree code the way it was implemented
